<a href="https://colab.research.google.com/github/kairamilanifitria/Bootcamp-NLP/blob/main/11092024_Text_Classification_RNN_%26_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/dataset_spam.csv')
data.tail()

,content,label
6663,f a n s b e t i n g agent bola terpercaya di i...,spam
6664,pasti ada campur tangan ulah orang dalam karen...,normal
6665,mauuu puunyaa kullittt puutiiihh secaaraa inns...,spam
6666,bagi aku kaaa dikit aja,normal
6667,mila sipitt allah tuh benci sama orang yg meru...,normal


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6668 entries, 0 to 6667
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  6668 non-null   object
 1   label    6668 non-null   object
dtypes: object(2)
memory usage: 104.3+ KB


In [8]:
X = data.content
Y = data.label
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
Y = Y.reshape(-1,1)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [10]:
# conversi teks ke sequence
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [11]:
# define RNN
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

In [12]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)                  │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 150, 50)             │          50,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          29,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC1 (Dense)                          │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_layer (Dense)                    │ (None, 1)                   │             257 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 96,337 (376.32 KB)

 Trainable params: 96,337 (376.32 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(sequences_matrix, Y_train,batch_size=128, epochs=10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.8256 - loss: 0.4428 - val_accuracy: 0.9047 - val_loss: 0.3216
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 198ms/step - accuracy: 0.9090 - loss: 0.3097 - val_accuracy: 0.9283 - val_loss: 0.2720
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 176ms/step - accuracy: 0.9382 - loss: 0.2342 - val_accuracy: 0.9422 - val_loss: 0.2493
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 165ms/step - accuracy: 0.9537 - loss: 0.1866 - val_accuracy: 0.9529 - val_loss: 0.2128
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 228ms/step - accuracy: 0.9531 - loss: 0.1646 - val_accuracy: 0.9465 - val_loss: 0.1480
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 176ms/step - accuracy: 0.9601 - loss: 0.1350 - val_accuracy: 0.9604 - val_loss: 0.1383
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 182ms/step - accuracy: 0.9644 - loss: 0.1224 - val_accuracy: 0.9475 - val_loss: 0.1717


In [14]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len)

In [15]:
accr = model.evaluate(test_sequences_matrix, Y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9498 - loss: 0.1864


In [16]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.191
  Accuracy: 0.944


In [17]:
Y_train_pred = model.predict(sequences_matrix)

146/146 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step


In [18]:
Y_test_pred = model.predict(test_sequences_matrix)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [23]:
Y_train_class = Y_train.tolist()

In [22]:
Y_train_pred_class = []
for i in range (len(Y_train_pred)):
  if Y_train_pred[i] < 0.5:
    Y_train_pred_class.append(1)
  else:
    Y_train_pred_class.append(0)


In [25]:
from sklearn.metrics import classification_report
print(classification_report(Y_train_class, Y_train_pred_class ))

              precision    recall  f1-score   support

           0       0.02      0.00      0.00      4016
           1       0.04      0.28      0.07       651

    accuracy                           0.04      4667
   macro avg       0.03      0.14      0.04      4667
weighted avg       0.02      0.04      0.01      4667



In [26]:
Y_test_class = Y_test.tolist()
Y_test_pred_class = []
for i in range (len(Y_test_pred)):
  if Y_test_pred[i] < 0.5:
    Y_test_pred_class.append(1)
  else:
    Y_test_pred_class.append(0)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_class, Y_test_pred_class ))

              precision    recall  f1-score   support

           0       0.02      0.00      0.00      1687
           1       0.06      0.34      0.10       314

    accuracy                           0.06      2001
   macro avg       0.04      0.17      0.05      2001
weighted avg       0.03      0.06      0.02      2001

